In [1]:
from IPython.display import IFrame

IFrame('Boyce_12e_PPT_ch03_8.pdf', width=1000, height=700)

- No forcing function --> 3.7 notice the omega value 

- tune frequency towards certain omega 

- change focus towards resonance

In [9]:
from demo import run_demo
run_demo()

Play(value=0, interval=50, layout=Layout(display='none'), max=599)

Output()